In [1]:
import numpy as np
import pandas as pd
import sys
from tqdm.notebook import tqdm
import networkx as nx
sys.path.append('..')

import torch
from torch.functional import F
import torch.nn as nn
from torch.distributions import Dirichlet

from torch_geometric.data import Data, DataLoader, Dataset
from torch_geometric.utils import from_networkx, to_networkx, degree
from torch_geometric.nn import Set2Set
from torch_geometric.utils.metric import accuracy, precision, f1_score
import torch_geometric.transforms as T

from models.graph_transformer.euclidean_graph_transformer import GraphTransformerEncoder
from models.deep_graph_infomax.infomax import SNInfomax

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score, precision_score
from sklearn.utils.class_weight import compute_class_weight

from utils.data_gen import load_prot_embs, SNDatasetInfomax, ucsv2graph_infomax, wcsv2graph_infomax, WSNDataset

dev = torch.device('cuda:0')

# Load the appropriate data

In [2]:
prot_embs, global_dict = load_prot_embs(512, norm=False)

In [3]:
all_semi = pd.read_csv('data/graph_info_df/file_info_weighted.csv')
all_semi = all_semi[all_semi.files_weighted != 'graphs_weighted/CPC019_MCF7_24H_BRD-K46373671-001-01-8_10/graph.csv']
u_path_list = all_semi.files_weighted.values
#all_moas = all_semi.moa_v1.values
#moa_v1 = all_moas[all_semi.moa_v1 != 'Unknown']
labels = all_semi.identifier.to_numpy().reshape(-1,1)

oh = OneHotEncoder()
labels = oh.fit_transform(labels).toarray()

train_data = WSNDataset(u_path_list, global_dict)

# Eval

In [4]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from karateclub.utils.treefeatures import WeisfeilerLehmanHashing
from karateclub.node_embedding import Role2Vec

In [5]:
fname = u_path_list[1700]

In [ ]:
train_loader = DataLoader(train_data, batch_size=8, num_workers=12, shuffle=True)
for u in tqdm(train_loader):
    print(u)